## Import packages

In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
import scipy.stats as ss
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from autoimpute.imputations.series import MultinomialLogisticImputer
from autoimpute.imputations.dataframe.base_imputer import MultinomialLogisticImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from mlxtend.evaluate import paired_ttest_5x2cv
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from collections import Counter
from sklearn import metrics

pd.set_option('display.max_columns', None)


## Read in csv data

In [3]:
df = pd.read_csv('adult.csv')
df = df.replace('?', np.nan)
#df.info()

# workclass, occupation, native.country have nan values

#### preprocessing

In [4]:
nominal_cols = ['workclass','marital.status','occupation','relationship','race','sex','income']
y_cols = ['income']
x_cols = ['age','workclass','fnlwgt','education.num','marital.status','occupation','relationship','race',
     'sex','capital','hours.per.week']

# Combine capital.loss and capital.gain
df['capital'] = df['capital.gain'] + df['capital.loss']

# remove unnecessary variables (education, native.country, capital.gain, capital.loss) native.country has too many values
df = df[['age','workclass','fnlwgt','education.num','marital.status','occupation','relationship','race',
        'sex','capital','hours.per.week','income']]


## 1. Imputation of data

In [5]:
# list of indices that contain nan values
wc_index = df[df['workclass'].isna()].index.tolist()
occ_index = df[df['occupation'].isna()].index.tolist()

#### Mean Imputation

In [6]:
df_mean_imputation = df.apply(lambda x: x.fillna(x.value_counts().index[0]))

#### Cold Deck Imputation

In [7]:
def cold_deck_impute(input_df, impute_col):
    model = KNeighborsClassifier(n_neighbors=1)
    df = input_df.copy()
    
    # df should have encoded/dummy categorical variables except for columns with nan
    x_cols = nominal_cols.copy()
    x_cols.remove(impute_col)
    ce_OHE = ce.OneHotEncoder(cols=x_cols)
    df_copy = ce_OHE.fit_transform(df)
    
    # imputation
    df_test = df_copy[df_copy[impute_col].isna()]
    df_train = df_copy.dropna()

    model.fit(df_train.loc[:, df_train.columns != impute_col], df_train[impute_col])
    df.loc[df[impute_col].isna(), impute_col] = model.predict(df_test.loc[:, df_test.columns != impute_col])
    return df

In [8]:
df_cold_deck = cold_deck_impute(df, 'workclass')
df_cold_deck = cold_deck_impute(df_cold_deck, 'occupation')

#### Regression imputation

In [9]:
def impute_regression(input_df, impute_col):
    imp = MultinomialLogisticImputer()
    df = input_df.copy()

    # df should have encoded/dummy categorical variables except for columns with nan
    x_cols = nominal_cols.copy()
    x_cols.remove(impute_col)
    ce_OHE = ce.OneHotEncoder(cols=x_cols)
    df_copy = ce_OHE.fit_transform(df)

    # imputation
    df_test = df_copy[df_copy[impute_col].isna()]
    df_train = df_copy.dropna()

    imp.fit(df_train.loc[:, df_train.columns != impute_col], df_train[impute_col])
    df.loc[df[impute_col].isna(), impute_col] = imp.impute(df_test.loc[:, df_test.columns != impute_col])

    return df

In [10]:
# workclass imputation
df_regression_imputation = impute_regression(df, 'workclass')

# occupation imputation
df_regression_imputation = impute_regression(df_regression_imputation, 'occupation')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


#### Check imputation results

In [11]:
# Check how many of each value was imputed here
test_df = df_mean_imputation
print(Counter(test_df.loc[wc_index, 'workclass']))
print(Counter(test_df.loc[occ_index, 'occupation']))

Counter({'Private': 1836})
Counter({'Prof-specialty': 1843})


### Encode Categorical data

In [12]:
# Deal with categorical variables
df_list = [df_mean_imputation, df_cold_deck, df_regression_imputation]
cols = ['workclass','marital.status','occupation','relationship','race','sex']
ce_OHE = ce.OneHotEncoder(cols=cols)

for i in range(len(df_list)):
    df_list[i] = ce_OHE.fit_transform(df_list[i])
df_dict = {'mean_impuation':df_list[0], 'cold deck':df_list[1], 'regression_imputation':df_list[2]}


In [13]:
df_dict['mean_impuation']

,age,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,workclass_7,workclass_8,fnlwgt,education.num,marital.status_1,marital.status_2,marital.status_3,marital.status_4,marital.status_5,marital.status_6,marital.status_7,occupation_1,occupation_2,occupation_3,occupation_4,occupation_5,occupation_6,occupation_7,occupation_8,occupation_9,occupation_10,occupation_11,occupation_12,occupation_13,occupation_14,relationship_1,relationship_2,relationship_3,relationship_4,relationship_5,relationship_6,race_1,race_2,race_3,race_4,race_5,sex_1,sex_2,capital,hours.per.week,income
0,90,1,0,0,0,0,0,0,0,77053,9,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,4356,40,<=50K
1,82,1,0,0,0,0,0,0,0,132870,9,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,4356,18,<=50K
2,66,1,0,0,0,0,0,0,0,186061,10,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,4356,40,<=50K
3,54,1,0,0,0,0,0,0,0,140359,4,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,3900,40,<=50K
4,41,1,0,0,0,0,0,0,0,264663,10,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,3900,40,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,1,0,0,0,0,0,0,0,310152,10,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,40,<=50K
32557,27,1,0,0,0,0,0,0,0,257302,12,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,38,<=50K
32558,40,1,0,0,0,0,0,0,0,154374,9,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,40,>50K
32559,58,1,0,0,0,0,0,0,0,151910,9,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,40,<=50K


## 2. Splitting data

In [14]:
df = df_dict['mean_impuation']

In [15]:
x = df.loc[:, df.columns != 'income']
y = df['income']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,random_state= 1)

In [16]:
# Feature Scaling (run this before classifier model)
sc = StandardScaler()
x_train[['age','hours.per.week','capital','fnlwgt']] = sc.fit_transform(x_train[['age','hours.per.week','capital','fnlwgt']])
x_test[['age','hours.per.week','capital','fnlwgt']] = sc.fit_transform(x_test[['age','hours.per.week','capital','fnlwgt']])

/var/folders/0d/6tkfrx094tb888cb90mgytt40000gp/T/ipykernel_99207/597751876.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train[['age','hours.per.week','capital','fnlwgt']] = sc.fit_transform(x_train[['age','hours.per.week','capital','fnlwgt']])
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/var/folders/0d/6tkfrx094tb888cb90mgytt40000gp

## Classifier Predictors

#### Decision Tree

In [17]:
dtc = DecisionTreeClassifier()
dtc.fit(x_train,y_train)

y_pred = dtc.predict(x_test)

In [18]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
cm

Accuracy: 0.8140641793336404


array([[4322,  621],
       [ 590,  980]])

#### Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(x_train,y_train)

y_pred = rfc.predict(x_test)

In [22]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
cm

Accuracy: 0.8582834331337326


array([[4588,  355],
       [ 568, 1002]])

#### Naive Bayes

##### Multinomial

In [140]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(x_train, y_train)

y_pred = mnb.predict(x_test)

In [141]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
cm

Accuracy: 0.7881160755412252


array([[4657,  286],
       [1094,  476]])

##### Gaussian

In [142]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(x_train, y_train)

y_pred = gnb.predict(x_test)

In [143]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
cm

Accuracy: 0.7950253339474896


array([[4831,  112],
       [1223,  347]])

#### Bayesian Network

In [144]:
# Can't use too many columns for Bayesian Network, must choose most significant columns
df = df_mean_imputation[['sex','education.num','marital.status','occupation','income']]
x_bayes = df.loc[:, df.columns != 'income']
y_bayes = df['income']
x_train, x_test, y_train, y_test = train_test_split(x_bayes, y_bayes, test_size = 0.2,random_state= 1)
train_data = x_train.join(y_train)

bn = BayesianNetwork([('education.num', 'income'),('marital.status', 'income'),('occupation', 'income'),('sex', 'income')])
bn.fit(train_data)
y_pred = bn.predict(x_test)

  0%|          | 0/840 [00:00<?, ?it/s]

In [145]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
cm

Accuracy: 0.8288039306003377


array([[4660,  283],
       [ 832,  738]])

### 3. Compare Predictive Accuracy

In [146]:
# evaluate DecisionTreeClassifier
dtc = DecisionTreeClassifier()
cv1 = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1)
scores1 = cross_val_score(dtc, x, y, scoring='accuracy', cv=cv1, n_jobs=-1)
print('DecisionTreeClassifier Mean Accuracy: %.3f (%.3f)' % (np.mean(scores1), np.std(scores1)))
# evaluate RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
cv2 = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1)
scores2 = cross_val_score(rfc, x, y, scoring='accuracy', cv=cv2, n_jobs=-1)
print('RandomForestClassifier Mean Accuracy: %.3f (%.3f)' % (np.mean(scores2), np.std(scores2)))
# evaluate MultinomialNB
mnb = MultinomialNB()
cv3 = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1)
scores3 = cross_val_score(mnb, x, y, scoring='accuracy', cv=cv3, n_jobs=-1)
print('MultinomialNB Mean Accuracy: %.3f (%.3f)' % (np.mean(scores3), np.std(scores3)))
# evaluate GaussianNB
gnb = GaussianNB()
cv4 = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1)
scores4 = cross_val_score(gnb, x, y, scoring='accuracy', cv=cv4, n_jobs=-1)
print('GaussianNB Mean Accuracy: %.3f (%.3f)' % (np.mean(scores4), np.std(scores4)))
# evaluate BayesianNetwork
df = df_mean_imputation[['sex','education.num','marital.status','occupation','income']]
x_bayes = df.loc[:, df.columns != 'income']
y_bayes = df['income']
x_train, x_test, y_train, y_test = train_test_split(x_bayes, y_bayes, test_size = 0.2,random_state= 1)
train_data = x_train.join(y_train)
bn = BayesianNetwork([('education.num', 'income'),('marital.status', 'income'),('occupation', 'income'),('sex', 'income')])
bn.fit(train_data)
y_pred = bn.predict(x_test)
print("BayesianNetwork Accuracy:",metrics.accuracy_score(y_test, y_pred))
print()

# Compare RandomForestClassifier and DecisionTreeClassifier
print('RandomForestClassifier and DecisionTreeClassifier comparison:')
t, p = paired_ttest_5x2cv(estimator1=dtc, estimator2=rfc, X=x, y=y, scoring='accuracy', random_seed=1)

# summarize RandomForestClassifier and DecisionTreeClassifier. They got the two highest.
print('P-value: %.3f, t-Statistic: %.3f' % (p, t))
# interpret the result
if p <= 0.05:
    print('Reject null hypothesis that the mean performance is the same. We can say RandomForestClassifier is significantly better.')
else:
    print('Fail to reject null hypothesis. RandomForestClassifier and DecisionTreeClassifier probably have the same performance')


DecisionTreeClassifier Mean Accuracy: 0.811 (0.003)
RandomForestClassifier Mean Accuracy: 0.852 (0.001)
MultinomialNB Mean Accuracy: 0.784 (0.002)
GaussianNB Mean Accuracy: 0.794 (0.002)


  0%|          | 0/840 [00:00<?, ?it/s]

BayesianNetwork Accuracy: 0.8288039306003377

RandomForestClassifier and DecisionTreeClassifier comparison:
P-value: 0.000, t-Statistic: -19.019
Reject null hypothesis that the mean performance is the same. We can say RandomForestClassifier is significantly better.


#### Tune RandomForest Parameters

In [264]:
from sklearn.model_selection import GridSearchCV

param_grid = [
{'n_estimators': [10, 25], 'max_features': [5, 10], 
 'max_depth': [10, 50, None], 'bootstrap': [True, False]}
]

grid_search_forest = GridSearchCV(rfc, param_grid, cv=10, scoring='accuracy')
grid_search_forest.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid=[{'bootstrap': [True, False],
                          'max_depth': [10, 50, None], 'max_features': [5, 10],
                          'n_estimators': [10, 25, 50]}],
             scoring='accuracy')

In [ ]:
#now let's how the RMSE changes for each parameter configuration
cvres = grid_search_forest.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
grid_search_forest.best_estimator_

In [ ]:
# Performance metrics
grid_best = grid_search_forest.best_estimator_.predict(x_test)
Print('Best accuracy from Grid Search:')
print("Accuracy:",metrics.accuracy_score(y_test, grid_best))
cm = confusion_matrix(y_test, y_pred)
cm

### 4. Compare Imputations and Classifiers

RandomForestClassifier seems to have the highest accuracy from the classifiers.
Below, we'll compare which imputation method leads to highest accuracy, by imputing with each method, then using the RandomForestClassifier on each imputed dataset.

In [149]:
def evaluate_accuracy(input_df, clf):
    df = input_df
    x = df.loc[:, df.columns != 'income']
    y = df['income']
    cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1)
    scores = cross_val_score(clf, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
    print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

In [199]:
clf_list = [DecisionTreeClassifier(), RandomForestClassifier(n_estimators=100), MultinomialNB(), GaussianNB()]
for clf in clf_list:
    for name, df in df_dict.items():
        print(clf, name)
        evaluate_accuracy(df, clf)
        print()

DecisionTreeClassifier() mean_impuation
Mean Accuracy: 0.810 (0.002)

DecisionTreeClassifier() cold deck
Mean Accuracy: 0.810 (0.003)

DecisionTreeClassifier() regression_imputation
Mean Accuracy: 0.811 (0.003)

RandomForestClassifier() mean_impuation
Mean Accuracy: 0.852 (0.001)

RandomForestClassifier() cold deck
Mean Accuracy: 0.852 (0.002)

RandomForestClassifier() regression_imputation
Mean Accuracy: 0.852 (0.002)

MultinomialNB() mean_impuation
Mean Accuracy: 0.784 (0.002)

MultinomialNB() cold deck
Mean Accuracy: 0.784 (0.002)

MultinomialNB() regression_imputation
Mean Accuracy: 0.784 (0.002)

GaussianNB() mean_impuation
Mean Accuracy: 0.794 (0.002)

GaussianNB() cold deck
Mean Accuracy: 0.794 (0.002)

GaussianNB() regression_imputation
Mean Accuracy: 0.794 (0.002)



In [197]:
# evaluate mean imputation on RandomForestClassifier
df = df_list[0]
x = df.loc[:, df.columns != 'income']
y = df['income']
rfc1 = RandomForestClassifier(n_estimators=100)
cv1 = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1)
scores1 = cross_val_score(rfc1, x, y, scoring='accuracy', cv=cv1, n_jobs=-1)
print('Mean impuatation RandomForestClassifier Mean Accuracy: %.3f (%.3f)' % (np.mean(scores1), np.std(scores1)))

# evaluate cold deck imputation on RandomForestClassifier
df = df_list[1]
x = df.loc[:, df.columns != 'income']
y = df['income']
rfc2 = RandomForestClassifier(n_estimators=100)
cv2 = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1)
scores2 = cross_val_score(rfc2, x, y, scoring='accuracy', cv=cv2, n_jobs=-1)
print('Cold deck imputation RandomForestClassifier Mean Accuracy: %.3f (%.3f)' % (np.mean(scores2), np.std(scores2)))

# evaluate mean imputation on RandomForestClassifier
df = df_list[2]
x = df.loc[:, df.columns != 'income']
y = df['income']
rfc3 = RandomForestClassifier(n_estimators=100)
cv3 = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1)
scores3 = cross_val_score(rfc3, x, y, scoring='accuracy', cv=cv3, n_jobs=-1)
print('Regression imputation RandomForestClassifier Mean Accuracy: %.3f (%.3f)' % (np.mean(scores3), np.std(scores3)))

print('RandomForestClassifier and DecisionTreeClassifier comparison:')
t, p = paired_ttest_5x2cv(estimator1=rfc3, estimator2=rfc2, X=x, y=y, scoring='accuracy', random_seed=1)
print('P-value: %.3f, t-Statistic: %.3f' % (p, t))
# interpret the result
if p <= 0.05:
    print('Reject null hypothesis that the mean performance is the same. We can say cold deck impuatation is better than regression impuation')
else:
    print('Fail to reject null hypothesis. cold deck and regression imputation lead to same accuracy in classifier')


Mean impuatation RandomForestClassifier Mean Accuracy: 0.853 (0.001)
Cold deck imputation RandomForestClassifier Mean Accuracy: 0.852 (0.001)
Regression imputation RandomForestClassifier Mean Accuracy: 0.852 (0.001)
RandomForestClassifier and DecisionTreeClassifier comparison:
P-value: 0.475, t-Statistic: -0.772
Fail to reject null hypothesis. cold deck and regression imputation lead to same accuracy in classifier


In [153]:
# View what values were imputed in each method

print('mean imputation, imputed values')
test_df = df_mean_imputation
print(Counter(test_df.loc[wc_index, 'workclass']))
print(Counter(test_df.loc[occ_index, 'occupation']))
print()
print('cold deck imputation, imputed values')
test_df = df_cold_deck
print(Counter(test_df.loc[wc_index, 'workclass']))
print(Counter(test_df.loc[occ_index, 'occupation']))
print()
print('regression imputation, imputed values')
test_df = df_regression_imputation
print(Counter(test_df.loc[wc_index, 'workclass']))
print(Counter(test_df.loc[occ_index, 'occupation']))

mean imputation, imputed values
Counter({'Private': 1836})
Counter({'Prof-specialty': 1843})

cold deck imputation, imputed values
Counter({'Private': 1368, 'Self-emp-not-inc': 147, 'Local-gov': 120, 'State-gov': 82, 'Federal-gov': 66, 'Self-emp-inc': 51, 'Without-pay': 1, 'Never-worked': 1})
Counter({'Other-service': 268, 'Adm-clerical': 259, 'Craft-repair': 229, 'Sales': 220, 'Exec-managerial': 214, 'Prof-specialty': 193, 'Machine-op-inspct': 131, 'Transport-moving': 88, 'Handlers-cleaners': 70, 'Tech-support': 59, 'Farming-fishing': 57, 'Protective-serv': 45, 'Priv-house-serv': 10})

regression imputation, imputed values
Counter({'Private': 1836})
Counter({'Craft-repair': 1696, 'Prof-specialty': 147})


#### Answer

All the different imputation methods, seem to have little to no difference in their effects on the Predictive accuracy of the classifier models. 
There is no significantly better impuatation method in terms of predictive accuracy.


We have compared the Predictive Accuracy of all the Classification models in Section 3. We have found that RandomForestClassifier is significantly the best classifier out of them.